In [ ]:
!wget https://www.dropbox.com/s/mwpt967ncsjp42a/diaries_revolution_labeled_ver1.csv?dl=0 -O diaries.csv 

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
df = pd.read_csv('diaries.csv')
df.head()

,Author,Date,Text,Позитив,Негатив,Беспокойство,Гнев,Грусть,Уверенность,Согласие
0,Михаил Богословский,1916-11-01,"Был в факультетском заседании, на котором Груш...",0.0,1.0,0.0,1.0,1.0,1.0,1.0
1,Лев Троцкий,1916-11-01,Утром со шпиком ходил на почту. После того пос...,1.0,1.0,1.0,0.0,1.0,1.0,1.0
2,Анатолий Савенко,1916-11-01,"Как жаль, что страна не узнает речи Милюкова. ...",0.0,1.0,1.0,0.0,1.0,0.0,0.0
3,Великий князь Николай Михайлович,1916-11-01,"Я долго колебался открыть тебе истину, но посл...",1.0,1.0,0.0,1.0,0.0,1.0,0.0
4,Императрица Александра Федоровна,1916-11-01,"Мой любимый, дорогой! Я очень огорчена тем, чт...",1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [ ]:
corpus = df['Text'].values
corpus

array(['Был в факультетском заседании, на котором Грушка предложил от имени факультета войти в Совет о возведении английского посла Бьюкенена в почетные члены Университета. Я не возражал, конечно, но мне это предложение было не по душе. Оно все же — порыв, а с англичанами, холодными, чопорными и расчетливыми, порывы недопустимы. Это не с французами.',
       'Утром со шпиком ходил на почту. После того посетили префекта. «Друг» оказался низкорослой сумрачной фигурой, южным флегматиком, из тех, про кого трудно сказать: облобызает или укусит. При мне ему принесли набор воровских инструментов, только что отобранных. Он любезно мне показал добычу, как бы свидетельствуя этим, что, по глубокому его мнению, у меня с подобными инструментами не может быть ничего общего. Тем не менее, он объявил мне, что я завтра же должен уезжать в одну из американских республик.В какую именно? Я ответил, что намерен ехать в Нью-Йорк. Префект как будто согласился, но, собственно говоря, лишь в принципе, так как 

In [ ]:
!pip install deeppavlov

In [ ]:
!python -m deeppavlov install morpho_ru_syntagrus_pymorphy

In [ ]:
from deeppavlov import build_model, configs
model = build_model(configs.morpho_tagger.UD2_0.morpho_ru_syntagrus_pymorphy, download=True)

In [ ]:
from tqdm import tqdm

In [ ]:
texts_tagged = []

# sentences = [['Я', 'шел', 'дом']]
for parse in model.batched_call(corpus, batch_size=16):
    text = []
    i = 0
    for word in parse.split('\n'): 
        tag_tokens = word.split('\t')
        if len(tag_tokens) == 1: 
            continue 
        text.append(tag_tokens[1] + '_' + tag_tokens[2])

    texts_tagged.append(' '.join(text))

In [ ]:
texts_tagged

['Был_AUX в_ADP факультетском_ADJ заседании_NOUN ,_PUNCT на_ADP котором_PRON Грушка_PROPN предложил_VERB от_ADP имени_NOUN факультета_NOUN войти_VERB в_ADP Совет_PROPN о_ADP возведении_NOUN английского_ADJ посла_NOUN Бьюкенена_PROPN в_ADP почетные_ADJ члены_NOUN Университета_PROPN ._PUNCT Я_PRON не_PART возражал_VERB ,_PUNCT конечно_ADV ,_PUNCT но_CCONJ мне_PRON это_DET предложение_NOUN было_AUX не_PART по_ADP душе_NOUN ._PUNCT Оно_PRON все_PART же_PART —_PUNCT порыв_NOUN ,_PUNCT а_CCONJ с_ADP англичанами_NOUN ,_PUNCT холодными_ADJ ,_PUNCT чопорными_ADJ и_CCONJ расчетливыми_ADJ ,_PUNCT порывы_NOUN недопустимы_ADJ ._PUNCT Это_PRON не_PART с_ADP французами_NOUN ._PUNCT',
 'Утром_NOUN со_ADP шпиком_NOUN ходил_VERB на_ADP почту_NOUN ._PUNCT После_ADP того_PRON посетили_VERB префекта_NOUN ._PUNCT «_PUNCT Друг_PROPN »_PUNCT оказался_VERB низкорослой_ADJ сумрачной_ADJ фигурой_NOUN ,_PUNCT южным_ADJ флегматиком_NOUN ,_PUNCT из_ADP тех_DET ,_PUNCT про_ADP кого_PRON трудно_ADJ сказать_VERB :_PUN

In [ ]:
import re 
import nltk 
import pymorphy2
from nltk.corpus import stopwords 

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
alnum_regex = re.compile("[А-Яа-я]+_[A-Za-z]+")

def find_words(text, regex = alnum_regex):
    tokens = regex.findall(text.lower())
    ans = []
    for w in tokens: 
        word, tag = w.split('_')
        if  word == 'не' or word.isalpha() and len(word) > 3: 
            ans.append(word + '_' + tag)
    return ' '.join(ans)

stopwords_set = set(stopwords.words('russian'))
stopwords_set.add('всё')
stopwords_set.add('это')
stopwords_set.add('мочь')
stopwords_set.add('очень')

def lemmatize(words, lemmer = morph, stopwords = stopwords_set):
    lemmas = []
    ans = []
    for w in words.split(): 
        word, tag = w.split('_')
        lemma = lemmer.parse(word)[0].normal_form
        if lemma == 'не' or lemma not in stopwords and lemma.isalpha(): 
            ans.append(lemma + '_' + tag.upper())

    return ans

def preprocess(text):
    return lemmatize(find_words(text))

In [ ]:
texts_tagged[0]

'Был_AUX в_ADP факультетском_ADJ заседании_NOUN ,_PUNCT на_ADP котором_PRON Грушка_PROPN предложил_VERB от_ADP имени_NOUN факультета_NOUN войти_VERB в_ADP Совет_PROPN о_ADP возведении_NOUN английского_ADJ посла_NOUN Бьюкенена_PROPN в_ADP почетные_ADJ члены_NOUN Университета_PROPN ._PUNCT Я_PRON не_PART возражал_VERB ,_PUNCT конечно_ADV ,_PUNCT но_CCONJ мне_PRON это_DET предложение_NOUN было_AUX не_PART по_ADP душе_NOUN ._PUNCT Оно_PRON все_PART же_PART —_PUNCT порыв_NOUN ,_PUNCT а_CCONJ с_ADP англичанами_NOUN ,_PUNCT холодными_ADJ ,_PUNCT чопорными_ADJ и_CCONJ расчетливыми_ADJ ,_PUNCT порывы_NOUN недопустимы_ADJ ._PUNCT Это_PRON не_PART с_ADP французами_NOUN ._PUNCT'

In [ ]:
corpus = list(map(lambda x: preprocess(x), texts_tagged))

In [ ]:
corpus = list(map(lambda x: ' '.join(x), corpus))
corpus[199]

'выехать_VERB киев_PROPN могилев_PROPN прежде_ADV командующий_NOUN гвардейский_ADJ корпус_NOUN великий_ADJ князь_NOUN павел_PROPN нанимать_VERB могилев_PROPN вернуться_VERB туда_ADV вместе_ADV свита_VERB девочка_NOUN устроить_VERB квартира_NOUN вагон_NOUN император_NOUN могилев_PROPN приехать_VERB повидаться_VERB императрица_NOUN ребёнок_NOUN'

In [ ]:
df['Text'] = corpus

In [ ]:
!pip uninstall pandas
!pip install pandas==1.1.5

In [ ]:
import pandas as pd

In [ ]:
df.to_csv('diaries_POS_tagged.csv', index=False)

In [ ]:
from google.colab import files

files.download('diaries_POS_tagged.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>